
## Introduction
WE are using Pandas to examine the auth.log of a server to check for intrusions. 

Pandas is used to manipulate training data before it is fed into a training model. 

Pandas (Python Data Analysis Library) "is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language."
___

This made up of data from Secrepo http://www.secrepo.com/ which contains Samples of Security Related Data

In [4]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns',5)

### File Input (CSV)
This next cell does a couple of things, first it imports pandas so we can create a dataframe, and then it reads our newly created file from above into memory. You can see the separator is specified to "\t" because Bro produces tab-delimited files by default. In this case we've also specified what we should call the columns in the dataframe.

In [9]:
logfile = 'conn_sample.log'
import pandas as pd
conn_df = pd.read_csv(logfile, sep="\t", header=None, names=['ts','uid','id.orig_h','id.orig_p','id.resp_h','id.resp_p','proto','service','duration','orig_bytes','resp_bytes','conn_state','local_orig','missed_bytes','history','orig_pkts','orig_ip_bytes','resp_pkts','resp_ip_bytes','tunnel_parents','threat','sample'])

### Verifying Input
Now (in theory) the contents of the file should be in a nicely laid-out dataframe.

For this next exercise, experiment with calling the **head()** and **tail()** method to see the values at the beginning and end of the dataframe. You can also pass a number to **head()** and **tail()** to specify the number of lines you want to see. Remember to click *play* or press *[Shift]+[Enter]* to execute the code in the cell after you change it.

In [10]:
conn_df.head()

,ts,uid,...,threat,sample
0,1.331901e+09,CX7YFb33Wlx68VyY3i,...,NaN,NaN
1,1.331901e+09,C3Ue4gd3oYXuNjkH2,...,NaN,NaN
2,1.331901e+09,C1UpLh1IiDGw4XKit7,...,NaN,NaN
3,1.331901e+09,CcWO2OX2RDWuSgf5c,...,NaN,NaN
4,1.331901e+09,COMNRodQHYvrp8949,...,NaN,NaN


### list column Headers or column names

In [31]:
conn_df.columns.values

array(['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p',
       'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes',
       'conn_state', 'local_orig', 'missed_bytes', 'history', 'orig_pkts',
       'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'tunnel_parents'], dtype=object)

### Data Summarization
Now create a new cell below this one. This can be accomplished by clicking on this cell once, and then clicking the *+* icon towards the top or selecting *Insert* from above and then selecting *Insert Cell Below*. After creating the new cell, it's time to learn about the **describe()** method that can be called on dataframes. This will give you a numeric summarization of all columns that contain numbers.

Try it out!

In [12]:
conn_df.describe()

,ts,id.orig_p,...,threat,sample
count,2.269400e+04,22694.000000,...,0.0,0.0
mean,1.331949e+09,42876.368335,...,NaN,NaN
std,4.280920e+04,15195.061536,...,NaN,NaN
min,1.331901e+09,3.000000,...,NaN,NaN
25%,1.331908e+09,36291.000000,...,NaN,NaN
50%,1.331928e+09,44317.000000,...,NaN,NaN
75%,1.331997e+09,54416.000000,...,NaN,NaN
max,1.332018e+09,65514.000000,...,NaN,NaN


### Data Types
Wait a second, isn't the ts column supposed to be a timestamp? Perhaps this column would be better suited as a time data type vs. a number.

Run the cell below to see what type of information Python stored in each column.

In [13]:
conn_df.dtypes

ts                float64
uid                object
id.orig_h          object
id.orig_p           int64
id.resp_h          object
                   ...   
resp_pkts           int64
resp_ip_bytes       int64
tunnel_parents     object
threat            float64
sample            float64
Length: 22, dtype: object

### Converting Column Types
Time to change the ts column to a datetime object! We will accomplish that by using a simple function provided called *to_datetime()*. The cell below runs this function on the ts column (what should be a time stamp), and then re-assigns this column back to the dataframe in the same place. A new timestamp column could have been added to the dataframe as well so both the float value and the datetime object columns are present.

Run the cell below to convert the column type.

In [15]:
from datetime import datetime
conn_df['ts'] = [datetime.fromtimestamp(float(date)) for date in conn_df['ts'].values]

In [16]:
conn_df.dtypes

ts                datetime64[ns]
uid                       object
id.orig_h                 object
id.orig_p                  int64
id.resp_h                 object
                       ...      
resp_pkts                  int64
resp_ip_bytes              int64
tunnel_parents            object
threat                   float64
sample                   float64
Length: 22, dtype: object

### Data Value Exploration
Verify that the conversion was successful. What is the datatype of the column now?

Scroll back up the page and note where you ran the **describe()** function. You'll see under the threat and sample columns there is likely the value of *NaN*. This stands for Not a Number and is a special value assigned to empty column values. There are a few ways to explore what values a column has. Two of these are **value_counts()** and **unique()**. 

Try them below on different columns. You can create new cells or if you want to get more than the last command worth of output you can put a print statement in front. 

What happens when you run them on a column with IPs (*id.orig_h, id.resp_h*)? What about sample or threat?

In [17]:
conn_df['sample'].unique()

array([ nan])

In [26]:
conn_df['service'].unique()

array(['http', '-', 'dns', 'ssl', 'ftp-data', 'smtp', 'ftp', 'ssh', 'dhcp'], dtype=object)

### Remove Columns
Another useful operation on a dataframe is removing and adding columns.  Since the threat and sample columns contain only *NaNs*, we can safely remove them and not impact any analysis that may be performed. 

Below the sample column is removed (dropped), add a similar line to drop the *threat* column and use a method from above to verify they are no longer in the dataframe.

In [19]:
conn_df.drop('sample', axis=1, inplace=True)

In [20]:
conn_df.drop('threat', axis=1, inplace=True)

### Row Selection

You can use column values to select rows from the dataframes (and even only view specific columns). First, select all rows that contain *SSL* traffic by running the cell below.

In [27]:
conn_df[conn_df['service'] == 'ssl'].head()

,ts,uid,...,resp_ip_bytes,tunnel_parents
694,2012-03-16 14:50:20.130,C5fIJY2gFFSRHNMT7j,...,1744,(empty)
3555,2012-03-16 15:24:03.670,C3sK6U2FVQTBlsmC6,...,2500,(empty)
3754,2012-03-16 15:27:07.740,C6wADf1dTu6ABEtij7,...,11352,(empty)
4029,2012-03-16 15:30:30.130,Ci3l194XvsiLaHpnd7,...,2532,(empty)
4114,2012-03-16 15:32:07.820,Chh2vv39qBfHXyWPtk,...,2605,(empty)


Next we can assign that result to a dataframe, and then look at all all the *SSL* connections that happen over ports other than 443.

In [24]:
ssl_df = conn_df[conn_df['service'] == 'ssl']
ssl_df[ssl_df['id.resp_p'] != 443].head()

,ts,uid,...,resp_ip_bytes,tunnel_parents
4258,2012-03-16 15:35:41.390,C5TAs84EMfq8FUReFj,...,2136,(empty)
9787,2012-03-16 20:22:13.470,CSDHCrxBUwxESlFq7,...,87196,(empty)
10043,2012-03-16 20:33:32.690,CPtZYf4ZGn24p9Nyo1,...,2005,(empty)
18116,2012-03-17 17:53:08.290,CWtid24fnLxpiTral8,...,2005,(empty)
18144,2012-03-17 17:53:53.390,CcJYzA61tfLhlaDS6,...,2005,(empty)


lets find all the SSH entries

In [30]:
http_df = conn_df[conn_df['service'] == 'ssh']
http_df['id.orig_h'].value_counts()

192.168.202.80     1
192.168.202.110    1
192.168.202.108    1
192.168.202.138    1
Name: id.orig_h, dtype: int64